In [1]:
import pandas as pd
from utils import get_dataset_dir

german_df = pd.read_csv(get_dataset_dir("financial_phrasebank_german"))                       
french_df = pd.read_csv(get_dataset_dir("financial_phrasebank_french"))
spanish_df = pd.read_csv(get_dataset_dir("financial_phrasebank_spanish"))


In [2]:
german_df = german_df.dropna()
french_df = french_df.dropna()
spanish_df = spanish_df.dropna()

In [3]:
german_df["lang"] = "de"
french_df["lang"] = "fr"
spanish_df["lang"] = "es"

In [4]:
final_dataset = pd.concat([german_df, french_df, spanish_df])
final_dataset.to_csv(get_dataset_dir("financial_phrasebank_multilingual"), index=False)

In [5]:
final_dataset.to_parquet(get_dataset_dir("financial_phrasebank_multilingual", ext="parquet"), index=False)